# Técnicas Avançadas de **Captura e Tratamento** de Dados

---
## Prof. Bernardo Alves Furtado
---
### MBA em Big Data, Business Analytics e Gestão de Negócios. @**IDP**
---
3 a 21 agosto  -- 21 horas/aula

In [ ]:
# Garantindo que exista a pasta para salvar nossos outputs.
import os
if not os.path.exists('data'):
    os.mkdir('data')

Retomando a partir da aula passada...
#### Link: https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/Aula2.ipynb

## Resolução exercício básico -- Aula 1
1. Leia a tabela disponível em:

https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/exemplo2.csv

5. Padronize os nomes, como feito para o exemplo1
6. Corrija os valores de impostos pagos.
7. Utilize o `describe()` ou outro comando nos impostos corrigidos e identifique a média de impostos pagos.
8. Utilize `sum(coluna)` para o valor total de impostos pagos.
9. O que mais é possível notar de estranho neste exemplo simples?
10. Alguém está familiarizado com o comando `value_counts()` do pandas? Ele pode ajudar a identificar se há valores repetidos (uma de vários jeitos possíveis).

## Respostas

In [ ]:
# Lendo o arquivo
import pandas as pd
file = 'https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/exemplo2.csv'
exercicio1 = pd.read_csv(file + '?raw=True')
exercicio1

In [ ]:
# Padronizando os nomes
exercicio1.nome = exercicio1.nome.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
exercicio1.nome = exercicio1.nome.str.lower()
exercicio1

In [ ]:
# Corrigindo valores pagos
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return x.replace('R$', '').replace('.', '').replace(',', '.')
    return x

exercicio1['impostos'] = exercicio1['impostos pagos'].apply(clean_currency).astype(float)
print(exercicio1.head())
print(exercicio1['impostos pagos'].dtype)

In [ ]:
# Média impostos pagos
exercicio1.impostos.mean()

In [ ]:
exercicio1.impostos.sum()

In [ ]:
# Valores esperados.
v, tt = '41.364,28', '206.821,40'
print(f'Valor médio impostos pagos: {v}.')
print(f'Total: {tt}.')

### Examinando os itens 9 e 10...

In [ ]:
for col in exercicio1.columns:
    print(f'{exercicio1[col].value_counts()}\n')


### Então? Procedimentos
1. Padronizar CPFs
2. Agregar (`groubpy`) por CPF, somando impostos pagos.
### Sempre muito cuidado com agregação: porque?

In [ ]:
print('')

In [ ]:
"""
Porque agregar sempre implica transformações nas variáveis.
O que fazer com os valores de cada uma das outras colunas no processo de agregação?
Somar, tirar a média? Contar o mais frequente?
"""

In [ ]:
# Padronizando CPFs
exercicio1.cpf = exercicio1.cpf.str.replace('-', '').str.replace('.', '').astype(str)
exercicio1

In [ ]:
ex1_agregado = exercicio1.groupby('cpf').agg('sum')
ex1_agregado


In [ ]:
ex1_agregado = exercicio1.groupby('cpf').agg('sum').reset_index()
ex1_agregado

## E agora? O que acontece se eu pedir para 'somar' e 'contar'?

In [ ]:
ex1_agregado = exercicio1.groupby('cpf').agg(['sum', 'count'])
ex1_agregado

---
# $$+$$: Exercícios com pandas. Revisão.

### Dados do Titanic. Análise livre

1. Pergunta: Passageiros de classes inferiores apresentam taxa de sobrevivência menor?

source: https://www.kaggle.com/c/titanic/data

# Kaggle

### "Inside Kaggle you’ll find all the code & data you need to do your data science work. Use over 50,000 public datasets and 400,000 public notebooks to conquer any analysis in no time."

### Dicionário

In [ ]:
'''
Data Dictionary
Variable	Definition	Key
survival 	Survival 	0 = No, 1 = Yes
pclass 	Ticket class 	1 = 1st, 2 = 2nd, 3 = 3rd
sex 	Sex
Age 	Age in years
sibsp 	# of siblings / spouses aboard the Titanic
parch 	# of parents / children aboard the Titanic
ticket 	Ticket number
fare 	Passenger fare
cabin 	Cabin number
embarked 	Port of Embarkation 	C = Cherbourg, Q = Queenstown, S = Southampton
Variable Notes

pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.
'''

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
# df = pd.read_csv("https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/titanic_test.csv?raw=True")
df.head()

# source: https://minerandodados.com.br/analise-de-dados-com-python-usando-pandas/

## Aperitivo. Missing dados


In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.Embarked.head()

In [ ]:
df.Cabin.value_counts()

In [ ]:
df.Age.mode()

In [ ]:
df.Fare.mean()

In [ ]:
values = {'Age': df.Age.mode()[0],
          'Cabin': 'SC',
          'Fare': df.Fare.mean()}

df.fillna(value=values, inplace=True)
df.info()

In [ ]:
# Replace: male, female
sex = {'male': 0, 'female': 1}
df['sex_dummy'] = df.Sex.replace(sex)
df.head()

In [ ]:
df.info()

In [ ]:
cols = [1, 2, 5, 9, 12]
df_simples = df.iloc[:, cols]
df_simples.head()

### Outro jeito de selecionar colunas?

In [ ]:
df_simples2 = df[['Survived', 'Pclass', 'Age', 'Fare', 'sex_dummy']]
df_simples2.head()

In [ ]:
corr = df_simples.corr()
corr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# plot the heatmap
sns.heatmap(corr.iloc[:],
            xticklabels=corr.columns,
            yticklabels=corr.columns,
            annot=True,
            cmap='viridis')

# Simple regression and output

In [ ]:
import statsmodels.formula.api as smf

def reg(y, data, colunas=''):
    """ Função que roda as regressões
        Entre com colunas e com base de dados """
    resultado = smf.ols(f"{y} ~  {colunas}", data=data).fit()
    sns.distplot(resultado.resid)
    plt.show()
    return resultado

res = reg('Survived', df_simples, 'Pclass + sex_dummy + Age')
print(res.summary())

---
## Nice output: com cara de paper

Outros resultados com boa apresentação, from Quantative Economics:

https://python.quantecon.org/ols.html


In [ ]:
from statsmodels.iolib.summary2 import summary_col

def print_reg3(m1):
    info_dict={'Log-likelihood': lambda x: f"{x.llf:.2f}",
               'R-squared Adj': lambda x: f"{x.rsquared_adj:.2f}",
               'AIC': lambda x: f"{x.aic:.2f}",
               'BIC': lambda x: f"{x.bic:.2f}",
               'No. observations': lambda x: f"{int(x.nobs):d}"}

    results_table = summary_col(results=[m1],
                                float_format='%0.2f',
                                stars=True,
                                model_names=['Model 1'],
                                info_dict=info_dict)

    results_table.add_title('Table - OLS Regressions')
    print(results_table)

    with open('data/nice_output.txt', 'w') as f:
        f.write(results_table.as_text())
    # return results_table

print_reg3(res)

---
# Exercício extra

0. Simples. Duas modificações apenas.
1. Modifique a função `print_reg3` para que imprima o resultado comparativo de três modelos.
2. Por exemplo, só com **Pclass**, adicionando **Age** e adicionando **sex_dummy**

# Formatos

# Dados tabulares

1. O que são?

In [ ]:
import pandas as pd

Rows, Columns = 'r', 'c'

Qual limitação imediata desse formato?

### Incluem RDBMSs -- Relational DataBase Management Systems

Incluem:

1. CVS -- comma-separated values
2. Qualquer TXT, na verdade ...
3. Planilhas (XLSX, ODS...)

---
# CSV ou TXT

* Delimited: claro, por um delimitador, usualmente
# ','

## Vantagens:
1. Aceito -- *read, write* -- por praticamente todos sistemas/databases/programas
2. Relativamente eficiente -- descrito por um autor como: ***fairly inneficient***

## Desvantagens:
1. Não contém de forma explícita o formato da informação `str, float, int`
2. Como também não o sistema decimal (ponto, vírgula, moedas)...


## Diferenças relevantes em relação ao EXCEL, por exemplo.

1. Alguém?
---

1. O tipo de informação (`float, date`) é (quase) sempre **inferred**
2. Não transparente.
3. Não contém os passos de transformação -- ou seja -- não consegue descrever as alterações feitas
4. Não mantém consistência entre linhas e colunas (enfatiza a questão visual)

---
## Parâmetros de leitura. Opcionais e padrão
# pd.read_csv(filepath)

1. `filepath`: 'endereço'. Obrigatório
2. `sep`: delimitador. Padrão: vírgula
3. `header`: Se utiliza primeira linha para nomes de colunas.
4. Padrão `header=0` (inferidas, a partir da 1a. linha de dados)
5. Utilize `header=None` para não inferir a 1a. linha como títulos de colunas.
6. Caso queira nomear colunas na leitura, use `names=['col1', 'col2']`.
7. Também é possível usar uma coluna específica como nome das colunas: `index_col='nome_col'` ou índice `int` da posição da coluna.
8. Limitar a leitura a algumas colunas: `usecols=['col1', 'col8']
#### Típico para o caso brasileiro
9. `encoding='latin-1'`
10. Para planilhas gigantes: `nrows=100`

source: Confira: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

---
# Fixed-width data

### Que não são delimitados por um separador, mas sim por posição específica na linha.

Vantagem. Compacto

### Exemplo de leitura de dados da amostra do Censo 2010
---

# SQL RDBMS
<span style="color:red">Structured</span> <span style="color:blue">Query</span> Language

Relational DataBase Management Systems

* 1 projeto (arquivo), várias tables. Múltiplas relações.

## Mais restritivas (estruturadas), porém, mais organizadas.

Funções mais relevantes:

1. GROUP BY (agregação)
2. JOIN (relacionamento)
3. WHERE (condição)

### https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html

In [ ]:
sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                id integer PRIMARY KEY,
                                name text NOT NULL,
                                priority integer,
                                status_id integer NOT NULL,
                                project_id integer NOT NULL,
                                begin_date text NOT NULL,
                                end_date text NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                            );"""

# Hierárquicos

![Tree](https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/tree.png?raw=True)

### ""... no specific limit on the depth or names of branches.""
(Mertz, 2021, p.71)

1. Diretórios
2. Redes
3. Grafos (em geral). Matrizes incompletas, esparsas


---
# JSON

### Língua franca HTML, CSS, dados

* JavaScript Object Notation

1. Fácil leitura humanos $$+$$ máquinas
2. Safe. Transparente (distinto de `pickle`, por exemplo)
3. Aceita `false, true, null, objects {}, arrays {}`
4. keys tem que ser **strings**

In [ ]:
import json

exemplo = """
    {"name": "Wes",
    "places_lived": ["United States", "Spain", "Germany"],
    "pet": null,
    "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
    {"name": "Katie", "age": 33, "pet": "Cisco"}]
    }
"""

# source: (McKinney, 2012, p.165)

In [ ]:
j = json.loads(exemplo)
print(type(j))
print(j)

In [ ]:
print(f'Keys: {j.keys()}')
print(f"Content of siblings: {j['siblings']}")
print(f"type {type(j['siblings'])}")

In [ ]:
print(f"{j['siblings'][0].keys()}")
print(f"{j['siblings'][0]['name']}")

### O que aconteceu com `"pet": null`?

In [ ]:
print(j['pet'])

* Observação: é possível usar [schema](https://json-schema.org/) para definir um formato específico de JSON válido para o seu projeto.

---
# Persistência JSON

In [ ]:
import json

# Saving
def save_json(ob, filename):
    with open(filename, 'w') as handler:
        json.dump(ob, handler)
    print(f'Saved! You can check {filename}')

# Loading
def load_json(filename):
    with open(filename, "r") as handler:
        ll = json.load(handler)
    print(f'A soma da lista lida é {sum(ll):,.0f}')

In [ ]:
l = [x ** 3 for x in range(5)]
f = 'data/my_json.json'
save_json(l, f)

In [ ]:
load_json(f)

## Exercício bastante simples

1. Crie uma lista qualquer em `python`.
2. Por exemplo, uma lista com 10 elementos, começando em 100 até 10, de 10 em 10.
*Dica*: a função range admite as opções `range(início, fim, step).
Se o step for negativo, a lista é decrescente, certo?
3. Salve em JSON
4. Load em JSON
5. Calcule a média.
